# Ensemble Kalman Filter Workflow

1. Available observation vector for $\vec{y_{k+1}}$, and calculate the observation ensemble from the directional uncertainties based on the wind speed and direction. To calculate the uncertainty for each point of the perimeter:
    1. Obtain the centroid $c_k = 1/M\sum{y_k}$ where $M$ is the number of points.
    2. Calculate vectors $\vec{v_k^m}$ for each point from the centroid to the point where $m$ is the $m^{th}$ point in the perimeter
    3. Obtain the wind speed and direction: $\vec{u_k}$ from the closest weather station to the centroid
    4. Uncertainty for each point $m$ is then $e_k^m = \alpha \left(1 - \vec{v_k^m} \cdot \vec{u_k}\right)$

2. Given a state vector $x_k$ and uncertainties around each point $P_k$, generate an ensemble 
$$X_k = (x_k^1, x_k^2, \cdots , x_k^N)$$
    1. For the first timepoint $k=0$, the state vector is the first observation $(x_0 = y_0)$, and the ensemble is obtained from directional uncertainties assigned based on the wind speed and direction
    2. **Question:** When creating the ensemble, do we preserve the mean $\overline{x_k}$?
    
3. From the ensemble $X_k$, advance for each state vector $X_{k+1} = F(X_k, u_k)$, where $u_k$ 

4. 

## Dependencies:

In [1]:
import sys
sys.path.append('/home/jovyan/farsite-devAPI/src/')
sys.path.append('/home/jovyan/python-helper/src/')

import geopandas as gpd
import pandas as pd
import numpy as np

from shapely.geometry import MultiPolygon, Polygon

import uuid
import random

import farsiteutils_v2 as futils
from kalmanutils import calculate_uncertainties_observed, interpolate_perimeter

from loggers import TimeEstimator

from matplotlib import pyplot as plt

## Default Values

In [2]:
SELECTED_FIRE_DEFAULT = 'Maria2019'
DATA_PATH = '/home/jovyan/data/'    # Folder to keep all the data while running the simulations

## Select a fire

```selected_fire``` variable holds the name of the fire selected. ```selected_fire = 'Maria2019'``` for now

In [3]:
fp = futils.FilePaths(DATA_PATH)
usr = futils.User(fp)

unique_desc = usr.db.gdfignitionAll['description'].unique()
print(f'Available fires are {unique_desc}')

selected_fire = 'Maria2019'
print(f'{selected_fire} is selected.\n')
if (unique_desc == selected_fire).any():
    print(f'{selected_fire} is found!')
else:
    raise ValueError(f'{selected_fire} not found in possible fire descriptions: {unique_desc}')

Database interaction not yet implemented. Use pickle file for dataframes instead!
Available fires are ['Maria2019' 'River2021' 'Bridge2021']
Maria2019 is selected.

Maria2019 is found!


In [4]:
from shapely.geometry import MultiPolygon, Polygon
import numpy as np

class State:
    def __init__(self, geom):
        self.geom = geom
        
        # Initialize
        self.vertices = self.calculate_vertices()
        self.lengths = self.calculate_lengths()
    def calculate_vertices(self):
        geom = self.geom
        if isinstance(geom, MultiPolygon):
            geompoly = geom.geoms[0]
        elif isinstance(geom, Polygon):
            geompoly = geom

        return np.array((geompoly.exterior.coords))
    
    def calculate_lengths(self):
        return np.sqrt((np.diff(self.vertices, axis=0)**2).sum(axis=1))
    
    def calculate_vector(self):
        # Returns column vector of the vertices (x0, y0, x1, y1, ...)
        return self.vertices.reshape(len(self.vertices)*2, 1)
    
    

In [5]:
def sample_geometry(current_state, uncertainties):
    
    maxlength = current_state.lengths.max()
    
    sampled_vertices = []
    
    # Choose a random direction
    theta = random.uniform(0,2*np.pi)

    for (x,y), sigma in zip(current_state.vertices, uncertainties):
        mu=0
        # randx = random.gauss(mu, sigma)
        # randy = random.gauss(mu, sigma)
        
        # Choose a normal random radius based on the given sigma
        radius = abs(random.gauss(mu, sigma))
        
        # Calculate x and y distance for the random
        randx = radius*np.cos(theta)
        randy = radius*np.sin(theta)
        
        sampled_vertices.append((x+randx, y+randy))

    sampled_vertices = np.array(sampled_vertices)
    # return Polygon(sampled_vertices).buffer(maxlength, join_style=1).buffer(-maxlength, join_style=1)
    return Polygon(sampled_vertices)


def interpolate_geometries(geoms, vertex_count = None):
    
    if vertex_count == None:
        vertex_count = 0
        for geom in geoms:
            if isinstance(geom, MultiPolygon):
                geom = geom.geoms[0]

            if vertex_count < len(geom.exterior.coords):
                vertex_count = len(geom.exterior.coords)

    interpolated_vertices = []
    for geom in geoms:
        if isinstance(geom, MultiPolygon):
            geom = geom.geoms[0]
        
        geom_state = State(geom)
        vertices = np.array(interpolate_perimeter(geom_state.calculate_vertices(), vertex_count))

        interpolated_vertices.append(vertices)
        
    return interpolated_vertices


def align_vertices(interpolated_vertices):

    time_estimator = TimeEstimator(len(interpolated_vertices)-1)

    minroll_lst = []
    
    aligned_vertices = [interpolated_vertices[0]]
    for i in range(len(interpolated_vertices)-1):
        print(time_estimator.info_str(i), end='\r', flush=True)

        right_vertices = interpolated_vertices[i+1]


        # Cycle right_vertices
        l2perroll = []
        for roll in range(len(interpolated_vertices)-1):
            diff = aligned_vertices[0] - right_vertices
            diff2sum = (diff[:,0]**2 + diff[:,1]**2).sum()

            # Calculate diff^2 in
            l2perroll.append(diff2sum)

            right_vertices = np.roll(right_vertices,1, axis=0)

        minroll_lst.append(np.argmin(l2perroll))

    for i in range(len(interpolated_vertices)-1):
        aligned_vertices.append(np.roll(interpolated_vertices[i+1], minroll_lst[i], axis=0))
    
    return aligned_vertices
## Check reverse direction too?

def create_ensembles(nsamples, current_state, windspeed, winddirection, filetype, objectid, datetime, description):
                     
    # uncertainty in both x and y for each vertex
    uncertainties = calculate_uncertainties_observed(current_state.vertices, windspeed, winddirection, scale=windspeed)*np.sqrt(current_state.lengths.sum())


    ignitions = {'igniteidx': [], 
                 'filetype': [],
                 'objectid': [],
                 'filepath': [],
                 'datetime': [],
                 'description': [],
                 'geometry': []}

    time_estimator = TimeEstimator(nsamples)
    for sample in range(nsamples):
        print(time_estimator.info_str(sample), end='\r', flush=True)

        ignitions['igniteidx'].append(uuid.uuid4().hex)
        ignitions['filetype'].append(filetype)
        ignitions['objectid'].append(f'{objectid}E{sample:04d}')

        geometry = sample_geometry(current_state, uncertainties)
        if isinstance(geometry, MultiPolygon):
            max_area = 0
            max_area_idx = 0
            for ix, geom in enumerate(geometry.geoms):
                if geom.area > max_area:
                    max_area = geom.area
                    max_area_idx = ix
            geometry = geometry.geoms[max_area_idx]

        ignitions['geometry'].append(geometry)

        ignitions['filepath'].append(f'{usr.db.fp.datadir}ignitions/{selected_fire}E{sample:04d}.shp')
        gpd.GeoDataFrame({'FID': [0], 'geometry': ignitions['geometry'][-1]}, 
                         crs='EPSG:5070').to_file(ignitions['filepath'][-1])

        ignitions['datetime'].append(datetime)
        ignitions['description'].append(description)
        
    return ignitions


def create_ensemble_matrix(geoms, vertex_count=None, aligned_geom=None):

    interpolated_vertices = interpolate_geometries(geoms, vertex_count=vertex_count)
    # Add first list of vertices from the state vector to align. align_vertices aligns all the perimeters w.r.t the first array
    if aligned_geom is not None:
        interpolated_vertices = [aligned_geom] + interpolated_vertices

    aligned_vertices = align_vertices(interpolated_vertices)
    if aligned_geom is not None:
        aligned_vertices = aligned_vertices[1:]
    else:
        aligned_geom = aligned_vertices[0]
    
    if vertex_count is None:
        vertex_count = aligned_vertices[0].shape[0]
    
    X = np.zeros((vertex_count*2, nsamples))
    for i, vertices in enumerate(aligned_vertices):
        X[:,i] = vertices.flatten()
        
    return X, aligned_geom, vertex_count


def state_to_ignitions(X_0, objectid_lst, datetime_lst, description_lst, filetype_lst, usr):

    geoms = [Polygon(zip(X_0[::2,i], X_0[1::2,i])).buffer(0) for i in range(X_0.shape[1])]

    ignitions = {'igniteidx': [],
                 'filetype': [],
                 'objectid': [],
                 'filepath': [],
                 'geometry': [],
                 'datetime': [],
                 'description': []}

    for sample, geometry in enumerate(geoms):
        filetype = filetype_lst[sample]
        datetime = datetime_lst[sample]
        description = description_lst[sample]
        objectid = objectid_lst[sample]
        
        
        ignitions['igniteidx'].append(uuid.uuid4().hex)
        ignitions['filetype'].append(filetype)
        ignitions['objectid'].append(f'{objectid}U{sample:04d}')

        if isinstance(geometry, MultiPolygon):
            max_area = 0
            max_area_idx = 0
            for ix, geom in enumerate(geometry.geoms):
                if geom.area > max_area:
                    max_area = geom.area
                    max_area_idx = ix
            geometry = geometry.geoms[max_area_idx]

        ignitions['geometry'].append(geometry)

        ignitions['filepath'].append(f'{usr.db.fp.datadir}ignitions/{description}U{sample:04d}.shp')
        gpd.GeoDataFrame({'FID': [0], 'geometry': ignitions['geometry'][-1]}, 
                         crs='EPSG:5070').to_file(ignitions['filepath'][-1])

        ignitions['datetime'].append(datetime)
        ignitions['description'].append(description)
    
    return ignitions

### Plotting functions

In [6]:
def compare_perimeters(alst, colors='rgbcmyk', showvertex = 0):
    fig, ax = plt.subplots(1,1, figsize=(4,2), dpi=200)
    
    for a,c in zip(alst, colors):
        if c == 'r':
            ax.plot(a[::2], a[1::2], c+'o', markersize=1)
        else:
            ax.plot(a[::2], a[1::2], c)
            
        ax.scatter(a[2*showvertex], a[2*showvertex + 1], facecolor=(0,0,0,0.2), edgecolor=(0,0,0,0.9))
        
    ax.set_ylim(1.502e6, 1.506e6)
    ax.set_xlim(-2.087e6, -2.081e6)
    
def compare_matrices(X, colors='rgbcmy', showvertex = 0, ax=None):
    
    if ax is None:
        fig, ax = plt.subplots(1,1, figsize=(4,2), dpi=200)
    
    for i in range(X.shape[1]):
        c = colors[i%len(colors)]
        ax.plot(X[::2,i], X[1::2,i], c)
        ax.scatter(X[2*showvertex,i], X[2*showvertex+1,i], facecolor=(0,0,0,0.2), edgecolor=(0,0,0,0.9))
        
    # ax.set_ylim(1.502e6, 1.506e6)
    # ax.set_xlim(-2.087e6, -2.081e6)
    
    return ax
    

In [7]:
class EnKF:
    # Initialize with the first observation
    # Windspeed and direction is given from PyLaski
    def __init__(self, observation: gpd.GeoSeries, windspeed: int, winddirection: int, nsamples: int, usr: futils.User):
        state_geom = gpd.read_file(observation['filepath'])['geometry'].set_crs(epsg=5070).iloc[0]
        
        
        self.state_datetime = observation['datetime']
        self.selected_fire = observation['description']
        self.igniteidx = observation.name
        self.current_state = State(state_geom)
        
        
        self.windspeed = windspeed
        self.winddirection = winddirection
        self.nsamples = nsamples
        self.usr = usr

        self.ignitions = create_ensembles(self.nsamples, self.current_state, self.windspeed, self.winddirection, 
                                     observation['filetype'], observation['objectid'],
                                     observation['datetime'], observation['description'])
        
        self.usr.db.gdfignition = self.usr.db.gdfignition.append(gpd.GeoDataFrame(self.ignitions).set_index('igniteidx').set_crs(epsg=5070))
        
        # Create ensemble matrix
        self.X_0, self.aligned_geom, self.vertex_count = create_ensemble_matrix(self.ignitions['geometry'])
        
    # Updates the state matrix
    # Need to first add the observation into the database (usr.db for now)
    def update(self, compareidx: str, lcpidx: str, barrieridx: str, observation: gpd.GeoSeries, windspeed: int, winddirection: int):
        # Advance the state in Farsite using the windspeed and direction
        inputData_lst = []
        mainapi_lst = []
        for igniteidx in self.ignitions['igniteidx']:
            inputData = {'description': self.selected_fire,
                         'igniteidx'  : igniteidx,
                         'compareidx' : compareidx,
                         'lcpidx'     : lcpidx,
                         'barrieridx' : barrieridx,

                         'windspeed': self.windspeed, 'winddirection': self.winddirection,
                         'relhumid': 90, 'temperature': 20}

            mainapi_lst.append(usr.calculatePerimeters(inputData))
            

        # from multiprocessing import Pool

        # numproc = 4
        # pool = Pool(processes=numproc)

        # Run for each FarsiteManual
        for mainapi in mainapi_lst:
            # pool.apply_async(farsite.run_command, callback=farsite.updatedb)
            mainapi.run_farsite()

        # pool.close()
        # pool.join()
        
        # Filter igniteidx->compareidx simulations only
        df = self.usr.db.gdfsimulation
        filtered_indices = df[df['igniteidx'].isin(self.ignitions['igniteidx']) & (df['compareidx'] == compareidx)].index
        
        self.Xt_1, self.aligned_geom, self.vertex_count = create_ensemble_matrix(df.loc[filtered_indices, 'geometry'], 
                                                                                 self.vertex_count, self.aligned_geom)
        
        
        
        # Calculate ensemble for the observation
        state_geom = gpd.read_file(observation['filepath'])['geometry'].set_crs(epsg=5070).iloc[0]
        igniteidx = observation.name
        state_datetime = observation['datetime']

        self.observed_state = State(state_geom)

        self.observed_ensemble = create_ensembles(self.nsamples, self.observed_state, windspeed, winddirection, 
                                     observation['filetype'], observation['objectid'],
                                     observation['datetime'], observation['description'])

        self.Y_1, self.aligned_geom, self.vertex_count = create_ensemble_matrix(self.observed_ensemble['geometry'], self.vertex_count, self.aligned_geom)
        
        ##############################
        # Calculate the Kalman Filter
        ###############################
        neighbor_count = 0
        invert = False
        normed = False

        xt_1 = self.Xt_1.mean(axis=1, keepdims=True)
        y_1 = self.Y_1.mean(axis=1, keepdims=True)

        Ex_1 = self.Xt_1 - xt_1.repeat(self.nsamples, axis=1)
        Ey_1 = self.Y_1 - y_1.repeat(self.nsamples, axis=1)


        connectivity = np.eye(2*self.vertex_count, k=0)
        for i in range(neighbor_count):
            connectivity += np.eye(2*self.vertex_count, k=2*(i+1)) + np.eye(2*self.vertex_count, k=-2*(i+1))


        Py_1 = 1/(nsamples-1)*np.matmul(Ey_1, Ey_1.T)*connectivity
        Pxy_1 = 1/(nsamples-1)*np.matmul(Ex_1, Ey_1.T)*connectivity

        if normed:
            max_Py = abs(Py_1).max()
            max_Pxy = abs(Pxy_1).max()

            Py_1 /= max_Py
            Pxy_1 /= max_Pxy
        
        if invert:
            Py_1inv = np.linalg.pinv(Py_1, hermitian=True)
        else:
            Py_1inv = np.zeros_like(Py_1)
            np.fill_diagonal(Py_1inv, 1/Py_1.diagonal())

        assert(np.allclose(np.matmul(Py_1inv, Py_1), np.eye(2*self.vertex_count))), 'Inverse calculation is incorrect'

        self.K = np.matmul(Pxy_1, Py_1inv)
        
        #### Update the state ensemble
        self.X_1 = self.Xt_1 + np.matmul(self.K, (self.Y_1 - self.Xt_1))
        
        ####################
        # Recursive updates
        ##################
        
        # Update ignitions
        self.X_0 = self.X_1
        self.ignitions = state_to_ignitions(self.X_0, self.observed_ensemble['objectid'], self.observed_ensemble['datetime'], 
                           self.observed_ensemble['description'], self.observed_ensemble['filetype'], self.usr)
        
        self.current_state = self.observed_state
        
        
        self.usr.db.gdfignition = pd.concat([self.usr.db.gdfignition, gpd.GeoDataFrame(self.ignitions, geometry='geometry', crs='EPSG:5070').set_index('igniteidx')])
        
        # x_1 = X_1.mean(axis=1, keepdims=True)
        
# ## obtain the igniteidx of the ignition
# ## The indices will be passed from the interface, which will include corresponding datetime, filepath etc.
# compareidx = '9f82e870591748a9a8a01346d174f2a1'    # t=1 observation
# lcpidx = '43b7f5db36994599861eec4849cc68fd'        # Index for Maria2019
# barrieridx = 'cb47616cd2dc4ccc8fd523bd3a5064bb'    # NoBarrier shapefile index        

In [8]:
initial = usr.db.gdfignition.iloc[0]
windspeed = 10
winddirection = 90
nsamples = 100

state = EnKF(initial, windspeed, winddirection, nsamples, usr)
X_0_lst = [state.X_0]

/tmp/ipykernel_12779/2103266057.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.usr.db.gdfignition = self.usr.db.gdfignition.append(gpd.GeoDataFrame(self.ignitions).set_index('igniteidx').set_crs(epsg=5070))


In [9]:
state.usr.db.gdfignition

,filetype,objectid,filepath,datetime,description,geometry
f23196b034474744bdca7df94b13e0f7,Ignition,21231,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 19:37:11,Maria2019,"POLYGON ((-2084217.148 1503560.461, -2084279.5..."
9f82e870591748a9a8a01346d174f2a1,Ignition,21232,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 19:58:22,Maria2019,"POLYGON ((-2082034.494 1504548.520, -2081987.6..."
1a6eb30ee33742e18fa6bb26678b5077,Ignition,21234,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 20:31:33,Maria2019,"POLYGON ((-2082034.494 1504548.520, -2081987.6..."
911f9b001c054b74a6ba70958f5f3c93,Ignition,21235,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 20:56:44,Maria2019,"POLYGON ((-2082396.219 1505343.939, -2082305.7..."
7957c2e604554199ba60c5b13aff2f40,Ignition,21236,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 21:22:55,Maria2019,"POLYGON ((-2082598.331 1505371.327, -2082338.5..."
...,...,...,...,...,...,...
1ac798a3159b4092ade1c9b5f9677160,Ignition,21231E0095,/home/jovyan/data/ignitions/Maria2019E0095.shp,2019-10-31 19:37:11,Maria2019,"POLYGON ((-2084071.794 1503549.108, -2084263.1..."
5a51257a0435478f8c636ad82f518ff0,Ignition,21231E0096,/home/jovyan/data/ignitions/Maria2019E0096.shp,2019-10-31 19:37:11,Maria2019,"POLYGON ((-2084263.617 1503607.701, -2084375.5..."
691815b4f4a64d61a47f4aa11ac89b2a,Ignition,21231E0097,/home/jovyan/data/ignitions/Maria2019E0097.shp,2019-10-31 19:37:11,Maria2019,"POLYGON ((-2084121.084 1503611.830, -2083911.6..."
9936a4f648784317b0473d3e47be02f0,Ignition,21231E0098,/home/jovyan/data/ignitions/Maria2019E0098.shp,2019-10-31 19:37:11,Maria2019,"POLYGON ((-2084249.542 1503665.107, -2084329.3..."


In [10]:
## obtain the igniteidx of the ignition
## The indices will be passed from the interface, which will include corresponding datetime, filepath etc.
lcpidx = '43b7f5db36994599861eec4849cc68fd'        # Index for Maria2019
barrieridx = 'cb47616cd2dc4ccc8fd523bd3a5064bb'    # NoBarrier shapefile index

observation = usr.db.gdfignition.iloc[1]
compareidx = observation.name    # t=1 observation
windspeed = 10
winddirection = 90

state.update(compareidx, lcpidx, barrieridx, observation, windspeed, winddirection)
X_0_lst.append(state.X_0)

{'description': 'Maria2019', 'igniteidx': '4445f02f26a842c587fe43076fd9a0fd', 'compareidx': '9f82e870591748a9a8a01346d174f2a1', 'lcpidx': '43b7f5db36994599861eec4849cc68fd', 'barrieridx': 'cb47616cd2dc4ccc8fd523bd3a5064bb', 'windspeed': 10, 'winddirection': 90, 'relhumid': 90, 'temperature': 20}
Choosing a perimeter from the database
{'description': 'Maria2019', 'igniteidx': '37d716cebdb540eda569d9d6075c7b16', 'compareidx': '9f82e870591748a9a8a01346d174f2a1', 'lcpidx': '43b7f5db36994599861eec4849cc68fd', 'barrieridx': 'cb47616cd2dc4ccc8fd523bd3a5064bb', 'windspeed': 10, 'winddirection': 90, 'relhumid': 90, 'temperature': 20}
Choosing a perimeter from the database
{'description': 'Maria2019', 'igniteidx': '049b662c3af64a05b14ad941cc826014', 'compareidx': '9f82e870591748a9a8a01346d174f2a1', 'lcpidx': '43b7f5db36994599861eec4849cc68fd', 'barrieridx': 'cb47616cd2dc4ccc8fd523bd3a5064bb', 'windspeed': 10, 'winddirection': 90, 'relhumid': 90, 'temperature': 20}
Choosing a perimeter from the d

/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_00801/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019E0001.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 12)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_00801/out


/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_00802/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019E0002.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 12)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_00802/out


...

/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_00898/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019E0098.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 12)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_00898/out


/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_00899/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019E0099.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 12)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_00899/out


/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


In [11]:
state.usr.db.gdfignition

,filetype,objectid,filepath,datetime,description,geometry
f23196b034474744bdca7df94b13e0f7,Ignition,21231,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 19:37:11,Maria2019,"POLYGON ((-2084217.148 1503560.461, -2084279.5..."
9f82e870591748a9a8a01346d174f2a1,Ignition,21232,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 19:58:22,Maria2019,"POLYGON ((-2082034.494 1504548.520, -2081987.6..."
1a6eb30ee33742e18fa6bb26678b5077,Ignition,21234,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 20:31:33,Maria2019,"POLYGON ((-2082034.494 1504548.520, -2081987.6..."
911f9b001c054b74a6ba70958f5f3c93,Ignition,21235,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 20:56:44,Maria2019,"POLYGON ((-2082396.219 1505343.939, -2082305.7..."
7957c2e604554199ba60c5b13aff2f40,Ignition,21236,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 21:22:55,Maria2019,"POLYGON ((-2082598.331 1505371.327, -2082338.5..."
...,...,...,...,...,...,...
c8d45476d3df46be9cc73c4761386270,Ignition,21232E0095U0095,/home/jovyan/data/ignitions/Maria2019U0095.shp,2019-10-31 19:58:22,Maria2019,"POLYGON ((-2084490.735 1503946.442, -2084516.2..."
b6320bd4872d415383b4b1623b6ca241,Ignition,21232E0096U0096,/home/jovyan/data/ignitions/Maria2019U0096.shp,2019-10-31 19:58:22,Maria2019,"POLYGON ((-2084730.165 1504420.591, -2084790.1..."
4643c796cd824d7eb931b9c5c0c3fa1e,Ignition,21232E0097U0097,/home/jovyan/data/ignitions/Maria2019U0097.shp,2019-10-31 19:58:22,Maria2019,"POLYGON ((-2084809.174 1504529.258, -2084776.4..."
ed7896822ff74750a073ade2b3bf1512,Ignition,21232E0098U0098,/home/jovyan/data/ignitions/Maria2019U0098.shp,2019-10-31 19:58:22,Maria2019,"POLYGON ((-2084645.308 1504662.967, -2084771.8..."


In [12]:
# ax = compare_matrices(state.X_0[:,10:20], showvertex=90)
# ax.plot(state.current_state.vertices[:,0], state.current_state.vertices[:,1],'k--')

In [13]:
observation = usr.db.gdfignition.iloc[2]
compareidx = observation.name    # t=1 observation
windspeed = 10
winddirection = 90

state.update(compareidx, lcpidx, barrieridx, observation, windspeed, winddirection)
X_0_lst.append(state.X_0)

{'description': 'Maria2019', 'igniteidx': 'e04956bd94d246218f50c02825680ae5', 'compareidx': '1a6eb30ee33742e18fa6bb26678b5077', 'lcpidx': '43b7f5db36994599861eec4849cc68fd', 'barrieridx': 'cb47616cd2dc4ccc8fd523bd3a5064bb', 'windspeed': 10, 'winddirection': 90, 'relhumid': 90, 'temperature': 20}
Choosing a perimeter from the database
{'description': 'Maria2019', 'igniteidx': '80d910de7fe14136bf98c4cce3082bbf', 'compareidx': '1a6eb30ee33742e18fa6bb26678b5077', 'lcpidx': '43b7f5db36994599861eec4849cc68fd', 'barrieridx': 'cb47616cd2dc4ccc8fd523bd3a5064bb', 'windspeed': 10, 'winddirection': 90, 'relhumid': 90, 'temperature': 20}
Choosing a perimeter from the database
{'description': 'Maria2019', 'igniteidx': 'b417d8713a3e41488f40a4cdc98d0ed6', 'compareidx': '1a6eb30ee33742e18fa6bb26678b5077', 'lcpidx': '43b7f5db36994599861eec4849cc68fd', 'barrieridx': 'cb47616cd2dc4ccc8fd523bd3a5064bb', 'windspeed': 10, 'winddirection': 90, 'relhumid': 90, 'temperature': 20}
Choosing a perimeter from the d

/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_00901/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019U0001.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 19)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_00901/out


/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_00902/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019U0002.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 19)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_00902/out


...

/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_00998/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019U0098.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 19)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_00998/out


/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_00999/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019U0099.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 19)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_00999/out


/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


In [14]:
state.usr.db.gdfignition

,filetype,objectid,filepath,datetime,description,geometry
f23196b034474744bdca7df94b13e0f7,Ignition,21231,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 19:37:11,Maria2019,"POLYGON ((-2084217.148 1503560.461, -2084279.5..."
9f82e870591748a9a8a01346d174f2a1,Ignition,21232,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 19:58:22,Maria2019,"POLYGON ((-2082034.494 1504548.520, -2081987.6..."
1a6eb30ee33742e18fa6bb26678b5077,Ignition,21234,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 20:31:33,Maria2019,"POLYGON ((-2082034.494 1504548.520, -2081987.6..."
911f9b001c054b74a6ba70958f5f3c93,Ignition,21235,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 20:56:44,Maria2019,"POLYGON ((-2082396.219 1505343.939, -2082305.7..."
7957c2e604554199ba60c5b13aff2f40,Ignition,21236,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 21:22:55,Maria2019,"POLYGON ((-2082598.331 1505371.327, -2082338.5..."
...,...,...,...,...,...,...
d4608b219d214ab49e80afdd362f0eca,Ignition,21234E0095U0095,/home/jovyan/data/ignitions/Maria2019U0095.shp,2019-10-31 20:31:33,Maria2019,"POLYGON ((-2084963.597 1503403.959, -2084991.5..."
a1ff9a289ffc4c6bb0f58277b96f5494,Ignition,21234E0096U0096,/home/jovyan/data/ignitions/Maria2019U0096.shp,2019-10-31 20:31:33,Maria2019,"POLYGON ((-2085368.854 1504982.879, -2085409.8..."
40fdfc3ec0a0405d9b7b0a592cb4f3c8,Ignition,21234E0097U0097,/home/jovyan/data/ignitions/Maria2019U0097.shp,2019-10-31 20:31:33,Maria2019,"POLYGON ((-2085015.632 1504659.704, -2084977.9..."
97f31c345f3f4ace8858dc9887311e06,Ignition,21234E0098U0098,/home/jovyan/data/ignitions/Maria2019U0098.shp,2019-10-31 20:31:33,Maria2019,"POLYGON ((-2085485.833 1504899.254, -2085663.1..."


In [15]:
# ax = compare_matrices(state.X_0[:,10:20], showvertex=90)
# ax.plot(state.current_state.vertices[:,0], state.current_state.vertices[:,1],'k--')

In [16]:
observation = usr.db.gdfignition.iloc[3]
compareidx = observation.name    # t=1 observation
windspeed = 10
winddirection = 90

state.update(compareidx, lcpidx, barrieridx, observation, windspeed, winddirection)
X_0_lst.append(state.X_0)

{'description': 'Maria2019', 'igniteidx': '1a45af0e78474861bee6ed368c21f79f', 'compareidx': '911f9b001c054b74a6ba70958f5f3c93', 'lcpidx': '43b7f5db36994599861eec4849cc68fd', 'barrieridx': 'cb47616cd2dc4ccc8fd523bd3a5064bb', 'windspeed': 10, 'winddirection': 90, 'relhumid': 90, 'temperature': 20}
Choosing a perimeter from the database
{'description': 'Maria2019', 'igniteidx': 'de3ca5ddbb1c40858ea5289afd4619ad', 'compareidx': '911f9b001c054b74a6ba70958f5f3c93', 'lcpidx': '43b7f5db36994599861eec4849cc68fd', 'barrieridx': 'cb47616cd2dc4ccc8fd523bd3a5064bb', 'windspeed': 10, 'winddirection': 90, 'relhumid': 90, 'temperature': 20}
Choosing a perimeter from the database
{'description': 'Maria2019', 'igniteidx': 'ce0a1f7fba3b40b48c4a7777bd861cd2', 'compareidx': '911f9b001c054b74a6ba70958f5f3c93', 'lcpidx': '43b7f5db36994599861eec4849cc68fd', 'barrieridx': 'cb47616cd2dc4ccc8fd523bd3a5064bb', 'windspeed': 10, 'winddirection': 90, 'relhumid': 90, 'temperature': 20}
Choosing a perimeter from the d

/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_01001/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019U0001.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 15)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_01001/out


/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_01002/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019U0002.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 15)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_01002/out


...

/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_01098/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019U0098.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 15)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_01098/out


/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


Farsite $Rev: 2395 $
Number of parallel threads = 4 (omp_get_num_procs = 32).
Loading lcp file for Farsite #1: /home/jovyan/farsite/inputs/landscapes/Maria2019.lcp
Loading inputs file for Farsite #1: /home/jovyan/data/20221209/Run_01099/config
Loading ignition file for Farsite #1: /home/jovyan/data/ignitions/Maria2019U0099.shp
Loading barrier file for Farsite #1: /home/jovyan/farsite/inputs/barriers/NoBarrier/NoBarrier.shp
Launching Farsite #1
Starting time step 0.000000 (maximum = 15)
Writing outputs for Farsite #1 to /home/jovyan/data/20221209/Run_01099/out


/home/jovyan/farsite-devAPI/src/farsiteutils_v2.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.gdfsimulation = self.gdfsimulation.append(gdfappend)


In [17]:
state.usr.db.gdfignition

,filetype,objectid,filepath,datetime,description,geometry
f23196b034474744bdca7df94b13e0f7,Ignition,21231,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 19:37:11,Maria2019,"POLYGON ((-2084217.148 1503560.461, -2084279.5..."
9f82e870591748a9a8a01346d174f2a1,Ignition,21232,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 19:58:22,Maria2019,"POLYGON ((-2082034.494 1504548.520, -2081987.6..."
1a6eb30ee33742e18fa6bb26678b5077,Ignition,21234,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 20:31:33,Maria2019,"POLYGON ((-2082034.494 1504548.520, -2081987.6..."
911f9b001c054b74a6ba70958f5f3c93,Ignition,21235,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 20:56:44,Maria2019,"POLYGON ((-2082396.219 1505343.939, -2082305.7..."
7957c2e604554199ba60c5b13aff2f40,Ignition,21236,/home/jovyan/farsite/inputs/maria_ignite/maria...,2019-10-31 21:22:55,Maria2019,"POLYGON ((-2082598.331 1505371.327, -2082338.5..."
...,...,...,...,...,...,...
bb6f46599e9546d69f99b452ddc7df10,Ignition,21235E0095U0095,/home/jovyan/data/ignitions/Maria2019U0095.shp,2019-10-31 20:56:44,Maria2019,"POLYGON ((-2085317.153 1504754.840, -2085106.1..."
538588cae9164de6990cf06895aae880,Ignition,21235E0096U0096,/home/jovyan/data/ignitions/Maria2019U0096.shp,2019-10-31 20:56:44,Maria2019,"POLYGON ((-2085007.821 1504097.108, -2085133.6..."
b14ffb3bcbf54cf6af4203b8554f2ed7,Ignition,21235E0097U0097,/home/jovyan/data/ignitions/Maria2019U0097.shp,2019-10-31 20:56:44,Maria2019,"POLYGON ((-2085370.147 1503552.896, -2085308.3..."
e751516bae364b7c85dbb45216edbdc9,Ignition,21235E0098U0098,/home/jovyan/data/ignitions/Maria2019U0098.shp,2019-10-31 20:56:44,Maria2019,"POLYGON ((-2085206.534 1505876.614, -2085110.1..."


#### Interpolate all the vertices

#### TODO: Vertices include an additional point when generated from geometry. Remove the repeat!

#### TODO: Check whether the aligned_vertices have the correct roll amount
1. Create matrices from the aligned_vertices, and form the $X_0$ ensemble state matrix
2. Using the observation $y_1$, update the state vector and obtain $X_1$

#### TODO - Question: Can we find a way to not interpolate the vertices
1. Without interpolation, we need to create a matrix that maps the observed vertices to the state vector